In [ ]:
datasets/dbpedia-url.txt
...........................................................................................................................................................................................................................................................................................................................................Absent urls: 16
Total urls: 331
Hits total for all urls: 111042
Avg. hits per url: 335.47

 datasets/n3-reuters-128-urls.txt
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Absent urls: 309
Total urls: 879
Hits total for all urls: 50506
Avg. hits per url: 57.46

 datasets/kore50-urls.txt
................................................................................................................................................Absent urls: 15
Total urls: 144
Hits total for all urls: 1525
Avg. hits per url: 10.59

In [ ]:
from time import time
for x in 3, 32, 10, 100:
    tic = time()
    make_queries(queries, parallel=x)
    print(x, time()-tic)

In [ ]:
for dataset_fpath in glob(datasets_fpaths):
    with open(dataset_fpath, "r") as in_f:
        queries = []
        for url in in_f:
            url = db2wiki(url.strip())
            query = 'origins:"{}"'.format(url)
            queries.append(query)
            
        print(queries)


In [8]:
russians = saved

In [ ]:
# search for entities
from glob import glob 
from traceback import format_exc
from os.path import join 

datasets_fpaths = "datasets/*txt"


def get_hits(diffbot_query_response):
    hits_num = diffbot_query_response["hits"]
    data = diffbot_query_response["data"]

    types = []
    for i, hit in enumerate(diffbot_query_response["data"]):
        types.append(hit["type"])

        
    return hits_num, types


saved = None

for dataset_fpath in glob(datasets_fpaths):
    print(dataset_fpath)
    total_hits = 0
    total_urls = 0
    total_absent = 0
    
    with open(dataset_fpath, "r") as in_f, open(dataset_fpath + ".absent", "w") as out_f:
        for url in in_f:
            try:
                url = dbpedia2wikipedia(url.strip())
                query = 'origins:"{}"'.format(url)
                r = make_query(query)
                db_response = json.loads(r.content)
                
                hits_num, types = get_hits(db_response)
                if url == "en.wikipedia.org/wiki/Russians": saved = db_response
                print(".", end="")  
                total_urls += 1
                if hits_num == 0:
                    total_absent += 1
                    out_f.write("{}\n".format(url))
                total_hits += hits_num
                
            except KeyboardInterrupt:
                break
            except:
                print(url, "error")
                print(format_exc())
        print("\n")
                                                
    print("Absent urls:", total_absent)
    print("Total urls:", total_urls)
    print("Hits total for all urls:", total_hits)
    print("Avg. hits per url: {:.2f}".format(float(total_hits)/total_urls))

datasets/dbpedia-url.txt
...........................................................................................................................................................................................................................................................................................................................................

Absent urls: 16
Total urls: 331
Hits total for all urls: 111042
Avg. hits per url: 335.47
datasets/n3-reuters-128-urls.txt
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
import requests
from pprint import pprint
import json 
import codecs


endpoint_diffbot = "http://kg.diffbot.com/kg/dql_endpoint"
 
ENTITY_TYPES = ["AdministrativeArea", "Article", "Corporation",
                    "DegreeEntity", "EducationMajorEntity", "EducationalInstitution",
                   "EmploymentCategory", "Image", "Intangible", "Landmark", "LocalBusiness",
                   "Miscellaneous", "Organization", "Person", "Place", "Product", "Role",
                   "Skill", "Video"]


EL_ENTITY_TYPES = ["AdministrativeArea", "Corporation", "EducationalInstitution",
                   "Landmark", "LocalBusiness", "Miscellaneous", "Organization", 
                   "Person", "Place", "Product"]

EL_POL_ENTITY_TYPES = ["AdministrativeArea", "Corporation", "EducationalInstitution",
                   "Landmark", "LocalBusiness", "Organization", 
                   "Person", "Place", "Product"]


def dbpedia2wikipedia(url, to_en=True):
    
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    
    if to_en:
        wiki_domain = "en.wikipedia.org/wiki/"
    else:
        wiki_domain = "wikipedia.org/wiki/"
        
    new_url = url.replace("dbpedia.org/resource/", wiki_domain)
    if new_url == url:
        new_url = url.replace("dbpedia.org/page/", wiki_domain)
        
    return new_url

token = None
def get_token():
    global token    
    if token:
        return token
    else:
        with open("../dbt", "r") as f:
            token = f.read().strip()
            return token

import grequests 

def make_queries(queries, parallel=32):    
    rs = []
    for query in queries:
        data = {
            "token": get_token(),
            "query": query,
            "type": "query"}

        rs.append(grequests.get(endpoint_diffbot, params=data))
    
    return grequests.map(rs, size=parallel)



def make_query(query):
    data = {
        "token": get_token(),
        "query": query,
        "type": "query"}
    r = requests.get(endpoint_diffbot, params=data)

    return r 


def save2json(output_fpath, r):
    with codecs.open(output_fpath, "w", "utf-8") as out:   
        out.write(json.dumps(json.loads(r.content)))
    print(output_fpath)
 

def query_and_save(query, output_fpath):
    r = make_query(query)
    save2json(output_fpath, r)

# DQL

In [ ]:
query_and_save('allUris:"barackobama.com"', "data/all-uris.json")
query_and_save('wikipediaUri:"en.wikipedia.org/wiki/Barack_Obama"', "data/wiki-uri.json")
query_and_save('allUris:"en.wikipedia.org/wiki/Barack\_Obama"', "data/all-uris-wiki.json")
query_and_save('origins:"en.wikipedia.org/wiki/Barack_Obama"', "data/origins.json")

In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [ ]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")